In [ ]:
import numpy as np
import scipy
import scipy.stats
import pandas as pd
import sunpy.map
import cv2
import skimage
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import seaborn_image as isns
sns.set_style("darkgrid")

%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from keras import layers

import autoenc_model as aem
import data_generator as dg

In [ ]:
# Read images
AIA193_2016 = sunpy.map.Map("data/aia_193A/2016-01*.fits")
AIA193_2016_matrix = np.array(list(d.data for d in AIA193_2016))


# Normalize images
min_values = AIA193_2016_matrix.min(axis=2).min(axis=1)
max_values = AIA193_2016_matrix.max(axis=2).max(axis=1)

rg = max_values - min_values
rg = np.maximum(rg, 1e-4*np.ones(rg.shape)).reshape([-1,1,1])

AIA193_2016_mnorm = (AIA193_2016_matrix-min_values.reshape([-1,1,1])) / rg


# Discretize images to detect faulty data
AIA193_2016_bytes = np.round(AIA193_2016_mnorm*255).astype(np.uint8)


# Calculate histograms
nbins = 10
hist_vectors = np.array([np.histogram(i.flatten(), bins=np.linspace(0,256,nbins))[0] for i in AIA193_2016_bytes])


# Cluster histograms
kmeans = KMeans(n_clusters=2, n_init=10)
kmeans.fit(hist_vectors/hist_vectors.max())

class1 = AIA193_2016_bytes[kmeans.labels_ == kmeans.labels_[0], :, :]
class2 = AIA193_2016_bytes[kmeans.labels_ != kmeans.labels_[0], :, :]

In [ ]:
isns.ImageGrid([i for i in class1[:9]], cmap="afmhot") # class2 is (probably) noise 

In [ ]:
clean_input = AIA193_2016_mnorm[kmeans.labels_ == kmeans.labels_[0], :, :]
clean_input_train, clean_input_test = train_test_split(clean_input, test_size=0.25)

In [ ]:
# input_img = keras.Input(shape=[256,256,1])
# latent_size = 64

# # lrelu = layers.LeakyReLU(0.2)
# optim = keras.optimizers.Adam(0.001)
# # optim = keras.optimizers.SGD(0.001)
# loss = "mse"


# #x = layers.Reshape([256,256,1])(input_img)
# x = input_img
# x = layers.Conv2D(8, 3, activation="relu", padding='same', strides=1)(x)
# x = layers.Conv2D(8, 3, activation="relu", padding='same', strides=1)(x)
# x = layers.MaxPooling2D()(x)

# x = layers.Conv2D(16, 3, activation="relu", padding='same', strides=1)(x)
# x = layers.Conv2D(16, 3, activation="relu", padding='same', strides=1)(x)
# x = layers.MaxPooling2D()(x)

# x = layers.Conv2D(32, 3, activation="relu", padding='same', strides=1)(x)
# x = layers.Conv2D(32, 3, activation="relu", padding='same', strides=1)(x)
# x = layers.MaxPooling2D()(x)

# x = layers.Flatten()(x)
# x = layers.Dropout(0.2)(x)
# x = layers.Dense(128, activation="relu")(x)
# x = layers.Dropout(0.1)(x)
# encoded = layers.Dense(code_size, activation="sigmoid")(x)

# x = layers.Dense(256, activation="relu")(encoded)

# x = layers.Reshape([8, 8, 4])(x)
# x = layers.UpSampling2D()(x)

# x = layers.Conv2D(64, 3, activation="relu", padding='same', strides=1)(x)
# x = layers.Conv2DTranspose(64, 3, activation="relu", padding='same', strides=2)(x)

# x = layers.Conv2D(32, 3, activation="relu", padding='same', strides=1)(x)
# x = layers.Conv2DTranspose(32, 3, activation="relu", padding='same', strides=2)(x)

# x = layers.Conv2D(16, 3, activation="relu", padding='same', strides=1)(x)
# x = layers.Conv2DTranspose(16, 3, activation="relu", padding='same', strides=2)(x)

# x = layers.Conv2D(16, 3, activation="relu", padding='same', strides=1)(x)
# x = layers.Conv2DTranspose(1, 3, activation="relu", padding='same', strides=2)(x)

# #decoded = layers.Reshape([256,256])(x)
# decoded = x


# encoder = keras.Model(input_img, encoded)
# decoder = keras.Model(encoded, decoded)
# print(encoder.summary())
# decoder.summary()


# autoencoder = keras.Model(input_img, decoder(encoder(input_img)))
# autoencoder.compile(loss=loss, optimizer=optim, metrics=["mae"])
# autoencoder.summary()

latent_size = 96
autoencoder, encoder, decoder = aem.gen_autoenc_model(latent_size, optim="adam", loss="mse")

In [ ]:
n_epochs = 200
batch_size = 256

history = autoencoder.fit(clean_input, epochs=n_epochs, batch_size=batch_size)

In [ ]:
clean_input.testing = True
autoencoder.evaluate(clean_input)
plt.plot(history.history['loss'], label='loss')
#plt.plot(history.history['val_loss'],label='val_loss')
plt.legend()
plt.show()

In [ ]:
import importlib
importlib.reload(dg)


#idx = np.random.permutation(80)[:5]

#example = clean_input_test[idx, :, :]
#example = clean_input[0][idx]
#print(example.shape)
clean_input = dg.SunImgAEGenerator("data/aia_193A/", max_values, min_values, 256, test_split=0.2, shuffle=True)
example = clean_input.sample(5)
code = encoder.predict(example)
reconstructed = decoder(code)

images_zipped = [i for i in zip(code, example, reconstructed)]
images = []

for i in images_zipped:
    images.append(np.reshape(i[0], [4,16]))
    images.append(np.squeeze(i[1]))
    images.append(np.squeeze(i[2]))
    

isns.ImageGrid(images, cmap="afmhot", vmin=0, vmax=1)

In [ ]:
code = np.random.normal(0, 1, [6, 64])
reconstructed = decoder(code)

images_zipped = [i for i in zip(code, reconstructed)]
images = []

for i in images_zipped:
    images.append(np.reshape(i[0], [4,16]))
    images.append(np.squeeze(i[1]))
    

isns.ImageGrid(images, cmap="afmhot", vmin=0, vmax=1, col_wrap=4)